In [59]:
import dataclasses
import functools
import itertools

import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import KetSugar as ks
import MaxLik as ml
import HammerProj as hp


samplings = np.load('probes_samplings.npz')
cal_sampl = samplings['n17']
test_sampl = samplings['n108']

cal_states = np.array([ks.BlochKet(theta, phi) for theta, phi in cal_sampl])
test_states = np.array([ks.BlochKet(theta, phi) for theta, phi in test_sampl])

SX = np.array([0,1,1,0]).reshape((2,2))
SZ = np.array([1,0,0,-1]).reshape((2,2))

def apply_noise(rho_in, p_x=0.0, p_z=0.0): 
    rho_x = ks.ApplyOp(rho_in, SX)
    rho_bit_flipped = p_x*rho_x/2 + (1-p_x/2)*rho_in
    rho_phase_flipped = (1-p_z/2)*rho_bit_flipped + p_z*ks.ApplyOp(rho_bit_flipped, SZ)/2
    return rho_phase_flipped


In [60]:
samplings.files

['n4', 'n6r', 'n6', 'n8', 'n12', 'n17', 'n26', 'n30', 'n32', 'n108']

In [ ]:
SAMPLING_KEY = 'n8'
ML_ITERS = 100
ML_THR = 1e-12

rotations_tomo_proj = np.array((
    (0,0),
    (np.pi,0),
    (np.pi/2, 0),
    (np.pi/2, np.pi),
    (np.pi/2, 1*np.pi/2),
    (np.pi/2, 3*np.pi/2)
))

def get_assumed_rpv(alpha, beta):
    coords = np.copy(rotations_tomo_proj)
    coords[:,0] *= 1+alpha
    coords[:,1] *= 1+beta
    coss = np.cos(coords[:,0]/2)
    sins = np.sin(coords[:,0]/2)
    exps = np.exp(1j*coords[:,1])
    pis = np.zeros((6,2,1), dtype=np.complex128)
    pis[:,0,0] = coss
    pis[:,1,0] = sins*exps
    return np.einsum('ki,kj->kij', pis.reshape((-1,2)), pis.conj().reshape((-1,2)))

def simulate_tomograms_pc_noiseless(probes_coord, errors, noise_x=0.0, noise_y=0.0):
    tomograms = []
    noise = 0.001 #in the sense of overall uniform purity drop
    alpha, beta = errors
    rpv_true = get_assumed_rpv(alpha, beta)
    for theta, phi in probes_coord:
        probe_ket = ks.BlochKet(theta, phi)
        probe_rho = ks.ketbra(probe_ket, probe_ket)
        probe_rho_noisy = apply_noise(probe_rho, noise_x, noise_y)                
        tomogram = np.array([ks.Overlap(probe_rho_noisy, proj).real for proj in rpv_true])         
        tomograms.append(tomogram*(1-noise) + noise)
    tomograms = np.array(tomograms)
    return tomograms

def general_cost_f(x, tomograms = None):    
    rpv = get_assumed_rpv(x[0], x[1])
    rho_gen = (ml.Reconstruct(tomogram, rpv, ML_ITERS, 1e-9, RhoPiVect = True, Renorm = True) for tomogram in tomograms)
    purities = np.array([ks.Purity(rho).real for rho in rho_gen])
    return -np.min(purities)

def get_cost_function(tomograms):
    return functools.partial(general_cost_f, tomograms = tomograms)


def find_errors(tomograms):
    TRIALS = 10
    cost_func = get_cost_function(tomograms)
    bnds = ((-0.3, 0.3), (-.3, .3))
    naive = cost_func((0,0))
    best_result_f = 0
    best_result_x = (0,0)
    for i in range(TRIALS):
        result = minimize(cost_func, x0 = (np.random.random(size=2)-0.5)*0.01, bounds=bnds, method='Nelder-Mead')
        if i == 0:
            best_result_f = result['fun']
            best_result_x = result['x']
        if result['fun'] < best_result_f:
            best_result_x = result['x']
            best_result_f = result['fun']
        if best_result_f < -0.99:
            break
    print(f'{naive:.3e}->{best_result_f}')
    return best_result_x, best_result_f

In [65]:
SAMPLES = 3
px = 0.001*0
py = 0.001*0

for i in range(SAMPLES):
    true = np.random.normal(np.array((0,0)), scale=0.05)
    tomos = simulate_tomograms_pc_noiseless(cal_sampl, true, px, py)
    resx, resf = find_errors(tomos)
    print(np.round(resx, 4))
    print(np.round(true, 4))
    print('----')



0 -0.9887178945510008
> 0 -0.9887178945510008
1 -0.9887674346547338
... 1 -0.9887674346547338
2 -0.9887670320505516
3 -0.9950368753336124
... 3 -0.9950368753336124
-9.670e-01->-0.9950368753336124
[ 0.0411 -0.04  ]
[ 0.0401 -0.0387]
----
0 -0.9417692570648128
> 0 -0.9417692570648128
1 -0.9383766197755845
2 -0.9736004224816377
... 2 -0.9736004224816377
3 -0.9948567339046135
... 3 -0.9948567339046135
-9.393e-01->-0.9948567339046135
[ 0.076  -0.0757]
[ 0.0743 -0.0728]
----
0 -0.993730328137741
> 0 -0.993730328137741
-8.896e-01->-0.993730328137741
[-0.0653  0.0127]
[-0.0612 -0.014 ]
----


In [66]:
SAMPLES = 3
#half percent
px = 0.005
pz = 0.005

for i in range(SAMPLES):
    true = np.random.normal(np.array((0,0)), scale=0.05)
    tomos = simulate_tomograms_pc_noiseless(cal_sampl, true, px, pz)
    resx, resf = find_errors(tomos)
    print(np.round(resx, 4))
    print(np.round(true, 4))
    print('----')



0 -0.9840751139329564
> 0 -0.9840751139329564
1 -0.9883484446778269
... 1 -0.9883484446778269
2 -0.9837349273678926
3 -0.9883615145831828
... 3 -0.9883615145831828
4 -0.9422340788148769
5 -0.9869650699435694
6 -0.983740421179476
7 -0.9883613728760563
8 -0.9841774644738115
9 -0.98462490677756
-9.405e-01->-0.9883615145831828
[-0.0323 -0.0216]
[-0.029  -0.0216]
----
0 -0.9855560088142665
> 0 -0.9855560088142665
1 -0.9855562415176887
... 1 -0.9855562415176887
2 -0.9817653914265587
3 -0.926941133096663
4 -0.9821586322765765
5 -0.9886063579129973
... 5 -0.9886063579129973
6 -0.9815670369604729
7 -0.9824591147902708
8 -0.9886063566815266
9 -0.9840284160830205
-9.264e-01->-0.9886063579129973
[-0.0404  0.0718]
[-0.0371  0.0273]
----
0 -0.9884504108343926
> 0 -0.9884504108343926
1 -0.9407548271305308
2 -0.9884505963866066
... 2 -0.9884505963866066
3 -0.9884508169696875
... 3 -0.9884508169696875
4 -0.9192936579714917
5 -0.9882509590343803
6 -0.9412859439133432
7 -0.9884477503806137
8 -0.988449967